In [ ]:
%reload_ext autoreload
%autoreload 3

import pytorch_lightning as pl
from dvclive.lightning import DVCLiveLogger
from pytorch_lightning.loggers import TensorBoardLogger

from src.data import OpenNeuroDataModule
from src.models import MultiHeadClassifier

In [ ]:
datamodule = OpenNeuroDataModule()

# this is a test
model = MultiHeadClassifier(n_inputs=10,
                            n_spatial_features=2,
                            n_temporal_features=10,
                            n_subjects=10)

trainer = pl.Trainer(
    max_epochs=1, accelerator='auto', log_every_n_steps=1,
    logger=[DVCLiveLogger(save_dvc_exp=True, report='md'),
            TensorBoardLogger('.')]
    )
trainer.fit(model, datamodule=datamodule)